In [2]:
"""For altering memory usage"""
from sage.interfaces.gap import get_gap_memory_pool_size, set_gap_memory_pool_size
n = get_gap_memory_pool_size()
set_gap_memory_pool_size(200*n)
n = get_gap_memory_pool_size()

In [1]:
"""List of indices of small groups by GAP library: [x,y]."""
small = []
for i in range(1,40):
    if len(gap.IdsOfAllSmallGroups(i+1)) <= 500:
        small += gap.IdsOfAllSmallGroups(i+1)
#         print(i+1)
    else:
        print(i+1)
print(len(small))

180


In [14]:
"""For new problem with general subgroups."""
class induce():
    """Function to return list of class sizes in default order."""
    def __init__(self):
        pass

    """Returns class sizes up to conjugacy"""
    def class_sizes(self, G):
        group_order = G.order()
        reps = G.conjugacy_classes_representatives()
        class_sizes = []
        for g in reps:
            class_sizes.append(group_order / G.centralizer(g).order())
#         print(class_sizes)
#         orders = []
#         for rep in reps:
#             orders.append(rep.order())
#         print(orders)
        return class_sizes

    """Function to find all subgroups. Returns list of subgroups."""
    def subgroups(self, G):
        subgroups = G.conjugacy_classes_subgroups()
#         print(subgroups)
        return subgroups

    """Function to find induced trivial character of each subgroup. Returns a list of lists."""
    def get_induced(self, subgroups, G):
        ind = []
        for subgroup in subgroups:
            ind.append(subgroup.trivial_character().induct(G))
#         print(ind)
        return ind

    """Forms the matrices that we use to solve the simultaneous equation later. The last entry is the trivial representation."""
    def mat(self, induced):
        matrix = []
        for char in induced:
            matrix.append(list(char.values()))
        neg = []
        for i in range(len(matrix[0])):
            neg.append(-1)
        matrix[-1] = neg
#         for line in matrix:
#             print(line)
        return matrix

    """Check if a subgroup H of G is core-free"""
    def core_free(self, G, H):
        if H.order() == 1: 
            return True
        core = self.subgroups(H)[1:]
        for subsub in core:
            if subsub.is_normal(G) == True:
                return False
        return True

    """Find all subgroups of G containing H (just the conjugates)"""
    def subgroups_containing(self, G, H, subgroups):
        subgroups_list = [] 
        h = H.order()
        H_conjugates = []
        H_normalizer = G.normalizer(H)
        H_norm_cosets = G.cosets(H_normalizer)
        for coset in H_norm_cosets:
            g = coset[0]
            H_conjugates.append(H.conjugate(g))
        for subgroup in subgroups:
            if subgroup.order() > h :
                sub = False
                i = len(H_conjugates)
                while sub == False and i != 0:
                    sub = H_conjugates[i-1].is_subgroup(subgroup)
                    i -= 1
                if sub == True:
                    subgroups_list.append(subgroup)
        return subgroups_list
    
    """Finding PHG"""
    def get_phg(self, G, subgroup,j):
        gens = []
        gens += subgroup.list()
        X = []
        elems = G.list()
        for g in elems:
            for h in subgroup.conjugate(g).list():
                if h not in X:
                    X.append(h)
        for g in elems:
            if g**j not in X:
                gens.append(g)
        hp_group = PermutationGroup(gens)
        print(hp_group.order(), G.order())
            

    """Test if there exists non-negative solutions. Prints the id of group that works and the ci, prints fail otherwise.
    phg option to test hypothesis
    """
    def test(self, G):
#         center = G.center()
#         if center.order() == G.order():
#             return ""
#         j = 0
#         for elem in center.list():
#             j = max(j, elem.order())
#         if is_prime(j) != True:
#             return ""
        a = self.class_sizes(G)
        b = self.subgroups(G)
        zeros = [0]*len(a)
        ret_list = ""
        for i in range(len(b)):
            cor = self.core_free(G, b[i])
            if cor == True:
                try:
#                     self.get_phg(G,b[i],j)
                    b_i = self.subgroups_containing(G, b[i], b)
                    c = self.get_induced(b_i, G)
                    d_i = self.mat(c)
                    e = Matrix(d_i).T
                    temp = b[i].trivial_character().induct(G).values()
#                     print(temp)
                    Y = vector(temp)
                    p = MixedIntegerLinearProgram(solver='ppl')
                    w = p.new_variable(nonnegative=True)
                    p.add_constraint(e*w == Y)
                    p.solve()
                    out = []
                    for j, v in (p.get_values(w).items()):
                        out.append('w_%s = %s' % (j, v))
                        """Prints subgroup and subgroup order."""
#                         if round(v,6) != 0:
#                             print(e.T[j])
#                             print(b_i[j], b_i[j].order())
                    """Prints the solution and SmallGroup Id"""
                    ret_list += str(gap.IdSmallGroup(b[i])[1]) + "," + str(gap.IdSmallGroup(b[i])[2])+"  "
                    print(str(gap.IdSmallGroup(G)[1]) + "," + str(gap.IdSmallGroup(G)[2]) +"    "+ str(gap.IdSmallGroup(b[i])[1]) + "," + str(gap.IdSmallGroup(b[i])[2]))
#                     print(e.T)
#                     print(b[i], b[i].order())
#                     print(out)
                except:
                    pass
#                     print('fail')
        return ret_list

In [15]:
"""Main function
If group is in id form.
https://www.gap-system.org/Manuals/pkg/SmallGrp-1.3/doc/chap1.html
This works on CoCalc/non-windows system with gap_packages library installed."""
if __name__ == "__main__":
    inducer = induce()
    groups = [[18,3]]
    out_a = []
    out_b = []
    for groupid in groups:
        print(groupid)
        G = gap.SmallGroup(groupid)
        ret = inducer.test(PermutationGroup(gap_group = G.AsPermGroup()))
        if len(ret) > 0:
            out_a.append(groupid)
            out_b.append(ret)
#         try:
#             inducer.test(PermutationGroup(gap_group = G.AsPermGroup()))
#         except:
#             pass
    for a in out_a:
        print(str(a[1])+","+str(a[2]))
    for b in out_b:
        print(b)
    """If group is in SAGE form.
    https://doc.sagemath.org/html/en/thematic_tutorials/group_theory.html#groups-of-small-order-as-permutation-groups"""
#     C5 = CyclicPermutationGroup(5)
#     G = C5.holomorph()
#     C2 = CyclicPermutationGroup(2)
#     groups = [direct_product_permgroups([C2,G])]
#     for group in groups:
#         try:
#             result = inducer.test(group)
#         except:
#             pass

[18, 3]
18 18
2 18
18 18
